In [1]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2021-02-14 07:16:04--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.83.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.83.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2021-02-14 07:16:05--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2b4c82fbc33bfdb4f8ab615660.dl.dropboxusercontent.com/cd/0/inline/BI7qMuJgCJqRWr3sK6Y03FJfwl1vVj-GO-maym42pJaJx0fm6vQ9kZxWZgzEEEhPEZU-gCjivKKL1AxWFp2xVhbDhXtogMFTfqtbq700doJkD43VuOPEsEwybhttDbu2tA0/file# [following]
--2021-02-14 07:16:06--  https://uc2b4c82fbc33bfdb4f8ab615660.dl.dropboxusercontent.com/cd/0/inline/BI7qMuJgCJqRWr3sK6Y03FJfwl1vVj-GO-maym42pJaJx0fm6vQ9kZxWZgzEEEhPEZU-gC

In [2]:
!unzip -q "/content/movie-reviews-dataset.zip"

In [3]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout , Bidirectional

In [4]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
    lambda text, label: (regex_replace(text, '<br />', ' '), label),
  )

In [5]:
train_data = prepareData('movie-reviews-dataset/train')
test_data = prepareData('movie-reviews-dataset/test')

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b"A lot of people hated this movie, but that I blame on two facts - 1 - They want it to be too much like the first couple of American Pie movies. and 2 - they are trying to take it too seriously.  This one I found was the best one out of all six, and I absolutely love Dwight. The plot is predictable, I'll say that, but then what teen comedy ISN'T? Road Trip, Dirty Deeds etc etc, they are all incredibly predictable, but still goddamn funny! I say this is worth watching, I love it, I find it hilarious. Just don't watch it while comparing it to the first ones because it's nothing alike.  After watching Naked Mile and this one, it became obvious that all the American Pie movies were about, and only about, the Stiflers. Which is FINE by me.  Watch. Enjoy. Love. xD"
1


In [6]:
model = Sequential()

In [7]:
model.add(Input(shape=(1,), dtype="string"))

In [8]:
max_tokens = 1000
max_len = 100
vectorize_layer = TextVectorization(
  max_tokens=max_tokens,
  output_mode="int",
  output_sequence_length=max_len,
)

In [9]:
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

model.add(vectorize_layer)

In [10]:
model.add(Embedding(max_tokens + 1, 128))

model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [11]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 [==============================] - 30s 27ms/step - loss: 0.5655 - accuracy: 0.6970
Epoch 2/10
782/782 [==============================] - 21s 26ms/step - loss: 0.4285 - accuracy: 0.8002
Epoch 3/10
782/782 [==============================] - 20s 26ms/step - loss: 0.4016 - accuracy: 0.8153
Epoch 4/10
782/782 [==============================] - 20s 26ms/step - loss: 0.3733 - accuracy: 0.8316
Epoch 5/10
782/782 [==============================] - 20s 26ms/step - loss: 0.3504 - accuracy: 0.8428
Epoch 6/10
782/782 [==============================] - 20s 26ms/step - loss: 0.3198 - accuracy: 0.8603
Epoch 7/10
782/782 [==============================] - 20s 26ms/step - loss: 0.2874 - accuracy: 0.8755
Epoch 8/10
782/782 [==============================] - 20s 26ms/step - loss: 0.2627 - accuracy: 0.8839
Epoch 9/10
782/782 [==============================] - 20s 26ms/step - loss: 0.2330 - accuracy: 0.8987
Epoch 10/10
782/782 [==============================] - 20s 26ms/step - loss: 0.212

In [13]:
model.evaluate(test_data)

782/782 [==============================] - 14s 17ms/step - loss: 0.6841 - accuracy: 0.7674


[0.6840915679931641, 0.7673599720001221]

In [14]:
text = "I loved the movie !"

In [15]:
model.predict([text])

array([[0.97130007]], dtype=float32)